In [3]:
import minerl
import gym

import sys
sys.path.append("../functions")

from Utils import load_data, save_data
from DQfDModel import build_model, load_model

import numpy as np
import random
from ActionCombos import load_combos, get_all_action_combos, int_action_to_dict, convert_match_actions, match_batch_actions

# from Buffer import add_transaction

c:\users\danie\appdata\local\programs\python\python36\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
model_weights_path = "../training/expert_model_1603572051_749999.h5"

In [5]:
action_combos_treechop_path = "../resources/action_combos_treechop.sav"

In [6]:
unique_angles_treechop_path = "../resources/unique_angles_treechop.sav"

In [7]:
action_keys = ['attack',
 'back',
 'camera',
 'forward',
 'jump',
 'left',
 'right',
 'sneak',
 'sprint']

In [8]:
n_action_navigate = 58368
n_action_treechop = 35840

In [9]:
model = load_model(n_action_treechop, model_weights_path)

In [10]:
action_combos = load_data(action_combos_treechop_path)

In [11]:
unique_angles_treechop = load_data(unique_angles_treechop_path)

In [10]:
def add_transition(replay_buffer, curr_states, actions, rewards, next_steps, dones, curr_state, empty_deque=False, ns=10, ns_gamma=0.99,is_done=True):
		ns_rew_sum = 0.
		trans = {}

		if empty_deque:
			while len(rewards) > 0:
				for i in range(len(rewards)):
					ns_rew_sum += rewards[i] * ns_gamma ** i
				
				trans['sample'] = [curr_states.popleft(), actions.popleft(), rewards.pop(0),
												next_steps.popleft(), is_done, ns_rew_sum, curr_state]
				replay_buffer.add_sample(trans)
		else:
			for i in range(ns):
				ns_rew_sum += rewards[i] * ns_gamma ** i
			trans['sample'] =  [curr_states.popleft(), actions.popleft(), rewards.pop(0),
								next_steps.popleft(), dones.popleft(), ns_rew_sum, curr_state]
			replay_buffer.add_sample(trans)

In [12]:
def interact(model, curr_obs, action_combos, n_action, epsilon_start = 0.99):
    empty_by_one = np.zeros((1, 1))
    empty_exp_action_by_one = np.zeros((1, 2))
    empty_action_len_by_one = np.zeros((1, n_action))
    
    epsilon = epsilon_start
    
    if random.random() <= epsilon:
        action_command = env.action_space.sample()
    else:
        temp_curr_obs = np.array(curr_obs)
        temp_curr_obs = temp_curr_obs.tolist()['pov'].reshape(1,temp_curr_obs.tolist()['pov'].shape[0], temp_curr_obs.tolist()['pov'].shape[1], temp_curr_obs.tolist()['pov'].shape[2])
        q, _, _ = model.predict([temp_curr_obs, temp_curr_obs,empty_by_one, empty_exp_action_by_one,empty_action_len_by_one])
        action_command = np.argmax(q)
        
    action_to_store = None
    action_to_take = None
   
    if (isinstance(action_command, (int, np.integer))):
        action_to_store = action_command

        combo = action_combos[action_command]
        action_to_take = int_action_to_dict(action_keys, action_combos[action_command])
    else:
        action_to_store = convert_match_actions(action_command, action_combos, unique_angles_treechop)
        action_to_take = int_action_to_dict(action_keys, action_combos[action_to_store])

    return action_to_take

In [13]:
env = gym.make("MineRLTreechop-v0")

In [13]:
env.make_interactive(port=6666, realtime=True)

In [12]:
!python -m minerl.interactor 6666

C:\Users\danie\AppData\Local\Programs\Python\Python36\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
No existing interactor found on port 31415. Starting a new interactor.
Traceback (most recent call last):
  File "C:\Users\danie\AppData\Local\Programs\Python\Python36\lib\site-packages\minerl\interactor\__main__.py", line 40, in run_interactor
    InstanceManager.add_existing_instance(interactor_port)
  File "C:\Users\danie\AppData\Local\Programs\Python\Python36\lib\site-packages\minerl\env\malmo.py", line 244, in add_existing_instance
    assert cls._is_port_taken(port), "No Malmo mod utilizing the port specified."
AssertionError: No Malmo mod utilizing the port specified.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\danie\AppData\Local\Programs\Python\Python36\lib\runpy.py", line 193, 

In [2]:
async def start_environment_interactive(environment, port):
    env = await gym.make(environment)
    
    
    return env

In [16]:
def run_agent_interactive(env, action_combos, n_action):
    curr_obs = env.reset()
    done = False
    prev_obs = []
    episode = 1
    while not done:
        action = interact(model, curr_obs, action_combos, n_action)

        curr_obs, reward, done, _ = env.step(action)
        
#         if episode > 10:
#             add_transition(rep_buffer, nstep_state_deque, nstep_action_deque, nstep_rew_list, nstep_nexts_deque,
#                            nstep_done_deque, curr_obs, False, nsteps, nstep_gamma)
            
        episode = episode + 1
        if done: print(done)

In [17]:
run_agent_interactive(env, action_combos, n_action_treechop)

True


In [ ]:
!python -m minerl.interactor 6666

In [1]:
!python --version

Python 3.7.9


In [15]:
import collections

In [21]:
d1 = collections.OrderedDict()

In [ ]:
[('attack', array(0)), ('back', array(1)), ('camera', array([-124.32714  ,    7.5763187], dtype=float32)),
 ('forward', array(0)), ('jump', array(1)), 
 ('left', array(0)), ('right', array(1)), ('sneak', array(1)), ('sprint', array(0))]

In [22]:
d1['attack'] = 0
d1['back'] = 0
d1['camera'] = [-124.32714  ,    7.5763187]
d1['forward'] = 0
d1['jump'] = 0
d1['left'] = 0
d1['right'] = 0
d1['sneak'] = 1
d1['sprint'] = 0

In [23]:
d1

OrderedDict([('attack', 0),
             ('back', 0),
             ('camera', [-124.32714, 7.5763187]),
             ('forward', 0),
             ('jump', 0),
             ('left', 0),
             ('right', 0),
             ('sneak', 1),
             ('sprint', 0)])

In [24]:
camera = d1['camera']

In [27]:
d1['camera'] = [0.0,0.0]

In [28]:
d1

OrderedDict([('attack', 0),
             ('back', 0),
             ('camera', [0.0, 0.0]),
             ('forward', 0),
             ('jump', 0),
             ('left', 0),
             ('right', 0),
             ('sneak', 1),
             ('sprint', 0)])